# CS109 Final Project: Predicting Offensive Playcalling in the NFL


## By Matt Goldberg, Adam Gilfix, Nathaniel Ver Steeg and Steven Rachesky


![Image of NFL](images/nfl_shield.png)

Imagine you are a Defensive Coordinator in the NFL. You'll likely find yourself in control of the defensive play-calling decisions, a difficult task especially with limited knowledge of the opponent's offensive tendencies or more specifically the exact play the offense will execute. Let's take Rob Ryan as an example (pictured below):


![Image of NFL](images/rob_ryan.jpg)


Rob began his NFL career as the Defensive Coordinator of the Oakland Raiders and more recently was fired as the Defensive Coordinator of the New Orleans Saints in the 2015 season. According to this [New York Times article](http://www.nytimes.com/2015/11/18/sports/football/rob-ryan-fired-by-saints-as-defense-struggles.html?_r=0), `"Among the 18 coaches who have been defensive coordinators for at least 10 seasons since 1990, Ryan’s defenses have allowed more points per game and more yards per game to opponents than any other."` It's clear that Ryan could have benefited from better play-calling decisions. In this project, we analyze all pass and run plays in the NFL from the 2003-2014 seasons and generate seven classifiers to predict whether the offense will run or pass. Hopefully Rob Ryan considers some of our best classifiers before he takes his [next job](http://profootballtalk.nbcsports.com/2015/12/09/rob-ryan-spotted-at-bills-practice-in-team-gear/) as a Defensive Coordinator, possibly with the Buffalo Bills under his brother, Rex Ryan. 

## Table of Contents
* [1. Scraping]()
* [2. Data Cleaning]()
* [3. Exploratory Data Analysis (EDA)]()
* [4. Model Development and Findings]()
* [5. Future Work and Conclusions]()

## 1. Scraping


In order to perform our analysis, we needed to scrape data on every NFL play from the 2002-2014 seasons. Although our analysis is based on the 2003-2014 seasons, we included the 2002 season in order to create lag features in our DataFrame such as each team's previous year pass percentage. We acquired this data from [Pro-Football Reference's](http://www.pro-football-reference.com) play-by-play account of each game using Matt Goldberg's pfr library. *Please refer to our [**Scraping Notebook**]() in order to view our exact process and scripts used in acquring the data.* This process resulted in a large DataFrame of **XXXX** number of samples and **YYYY** number of features. Some of the features we acquired are **...**, however, we realized we needed to infer more specific features in order to complete our analysis. We  abstracted these later features and discuss difficulties in the accuracy and cleaning of Pro-Football Reference's recount of each game in our [**Data Cleaning**]() section. 

## 2. Data Cleaning


## 3. Exploratory Data Analysis (EDA)

## 4. Model Development and Findings

## 5. Future Work and Conclusions